In [ ]:
!pip install sentence-transformers faiss-cpu transformers gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.4 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


In [ ]:
menu_items = [
    {"day": "Monday", "item": "Grilled Chicken Wrap", "price": 4.50, "protein": 28, "calories": 400},
    {"day": "Monday", "item": "Veggie Burger", "price": 3.80, "protein": 12, "calories": 350},
    {"day": "Tuesday", "item": "Tofu Salad Bowl", "price": 4.20, "protein": 15, "calories": 300},
    {"day": "Tuesday", "item": "Beef Burrito", "price": 5.50, "protein": 30, "calories": 600},
    {"day": "Wednesday", "item": "Cheese Pizza Slice", "price": 2.50, "protein": 10, "calories": 450},
]

In [ ]:
docs = [
    f"{m['item']} ({m['day']}) — ${m['price']}, {m['protein']}g protein, {m['calories']} calories"
    for m in menu_items
]

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = model.encode(docs)

In [ ]:
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(doc_embeddings))

In [ ]:
def recommend_meal(query, top_k=2):
    query_vec = model.encode([query])
    D, I = index.search(np.array(query_vec), top_k)  # distances + indices
    results = [docs[i] for i in I[0]]
    return results

# def recommend_meal(query, top_k=2):
#     query_vec = model.encode([query])
#     D, I = index.search(np.array(query_vec), top_k)
#     results = [docs[i] for i in I[0]]
#     return "\n\n".join([f"🥗 {r}" for r in results])

In [ ]:
print("Query: protein-heavy food under $5")
print("Recommendations:")
for r in recommend_meal("protein-heavy food under $5"):
    print("-", r)

print("\nQuery: cheap vegetarian option")
for r in recommend_meal("cheap vegetarian option"):
    print("-", r)

Query: protein-heavy food under $5
Recommendations:
- Veggie Burger (Monday) — $3.8, 12g protein, 350 calories
- Beef Burrito (Tuesday) — $5.5, 30g protein, 600 calories

Query: cheap vegetarian option
- Veggie Burger (Monday) — $3.8, 12g protein, 350 calories
- Beef Burrito (Tuesday) — $5.5, 30g protein, 600 calories


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.5 MB/s eta 0:00:00


In [ ]:
import streamlit as st

st.title("Meal FAQ Chatbot")
user_question = st.text_input("Ask your question:")
if user_question:
    for r in recommend_meal(user_question):
      st.write("Answer:", r)


2025-10-10 19:14:12.895 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-10 19:14:13.085 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-10-10 19:14:13.086 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-10 19:14:13.088 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-10 19:14:13.090 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-10 19:14:13.091 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-10 19:14:13.091 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-10 19:14:13.093 Thread 'MainThread': mi

In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=recommend_meal,
    inputs=gr.Textbox(label="Ask your meal question:"),
    outputs=gr.Textbox(label="Recommended Meals"),
    title="🍴 Cafeteria Meal Advisor",
    description="Ask things like: *'What should I eat today under $5?'* or *'Give me a protein-rich lunch option.'*"
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a8c98d800dc98b5203.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
